In [1]:
import papermill as pm
from joblib import Parallel, delayed

In [2]:
from glob import glob

In [3]:
projects = {'jupyter': ['repo2docker', 'notebook', 'nbconvert'],
            'jupyterhub': ['zero-to-jupyterhub-k8s', 'the-littlest-jupyterhub', 'jupyterhub',
                           'binderhub', 'binder', 'team-compass'],
            'jupyterlab': ["jupyterlab"]}

In [6]:
import shutil as sh
import os
if os.path.exists('../reports'):
    sh.rmtree('../reports/')
os.makedirs('../reports')

In [9]:
for org, repos in projects.items():
    print(org)
    for repo in repos:
        print('    '+repo)
        this_ntbk_params = '{}-{}'.format(org, repo)
        new_name = '../reports/viz_data_{}.ipynb'.format(this_ntbk_params)
        pm.execute_notebook('./viz_data.ipynb', new_name, parameters={'org': org, 'repo': repo, "plot_N": 20})

jupyter
    repo2docker


    notebook


    nbconvert



jupyterhub
    zero-to-jupyterhub-k8s



    the-littlest-jupyterhub



    jupyterhub



    binderhub



    binder



    team-compass



jupyterlab
    jupyterlab
